In [1]:
import ITensors:op
import Pkg; Pkg.add("HDF5")
import Pkg; Pkg.add("Plots")
import Pkg; Pkg.add("PyFormattedStrings")
import Pkg; Pkg.add("LinearAlgebra")
import Pkg; Pkg.add("LaTeXStrings")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
using ITensors, ITensorMPS
import ITensors:op
using HDF5
using Plots
using PyFormattedStrings
using LinearAlgebra

ITensors.disable_warn_order()

14

In [3]:
# construction of the MPO hamiltonian
#------------------------------------------
function Hamiltonian(sites)
    ops = OpSum()

    N = length(sites)
    # spin-spin interaction
    for k=1:N-1
        ops += J,   "Sz",k, "Sz",k+1
        ops += J/2, "S+",k, "S-",k+1
        ops += J/2, "S-",k, "S+",k+1
    end
    #make it a ring
    ops += J,   "Sz",N, "Sz",1
    ops += J/2, "S+",N, "S-",1
    ops += J/2, "S-",N, "S+",1

    # spin-B interaction
    for k=1:N
        ops += -B, "Sz",k
    end
    
    H = MPO(ops,sites)
    #--------------------------
    return H
end

Hamiltonian (generic function with 1 method)

In [4]:

#four point RDM of sites AB, CD
# AB adjacent, CD adjacent
#------------------------------------------
function get_4pdm(wf::MPS,locs)
    psi = copy(wf)
    A,C = locs

    orthogonalize!(psi,A)
    ket = psi[A]
    for k in A+1:C+1
        ket *= psi[k]
    end
    rho = prime(ket,"Site") * dag(ket)

    if C-A>2
        inds_list = inds(rho)
        trinds1 = collect(3 : C-A)
        trinds2 = trinds1[end] .+ trinds1 .+ 2
        for idx in 1:length(trinds1)
            rho = rho*delta(inds_list[trinds1[idx]],inds_list[trinds2[idx]])
        end
    end
    #-------------------
    return rho
end

get_4pdm (generic function with 1 method)

In [5]:
#   entanglement details between four sites
#-----------------------------------
function calc_LogNeg_4site(wf::MPS,locs)    
    psi = copy(wf)
    A,C = locs

    rho = get_4pdm(psi,[A,C])
    old = inds(rho)

    function get_LogNeg(rho,old,new)
        rho_PT = swapinds(rho,old,new)
        egn_val = calc_Eigenvalues(rho_PT)
        Neg =  abs(sum( [lam for lam in egn_val if lam<0] ))
        return log(1+2*Neg)
    end
        
    #   ABCD = 1234 
    #   ABCD = 5678

    # wrt A B C D
    new_A  = [old[5],old[2],old[3],old[4],old[1],old[6],old[7],old[8]]
    new_B  = [old[1],old[6],old[3],old[4],old[5],old[2],old[7],old[8]]
    new_C  = [old[1],old[2],old[7],old[4],old[5],old[6],old[3],old[8]]
    new_D  = [old[1],old[2],old[3],old[8],old[5],old[6],old[7],old[4]]

    # wrt AB AC AD
    new_AB = [old[5],old[6],old[3],old[4],old[1],old[2],old[7],old[8]]
    new_AC = [old[5],old[2],old[7],old[4],old[1],old[6],old[3],old[8]]
    new_AD = [old[5],old[2],old[3],old[8],old[1],old[6],old[7],old[4]]

    LogNeg = 1
    for new in [new_A,new_B,new_C,new_D,new_AB,new_AC,new_AD]
        LogNeg *= get_LogNeg(rho,old,new)
    end
    LogNeg = LogNeg^(1.0/7.0)

    #-----------------------------------
    return LogNeg
end

calc_LogNeg_4site (generic function with 1 method)

In [6]:
#   calc_Eigenvalues of density matrix
#-----------------------------------
function calc_Eigenvalues(density_matrix)
    egn_val,_ = eigen(density_matrix,ishermitian=true)
    #-----------------------------------
    return diag(array(egn_val))
end


#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_Norm(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    Norm = sum(egn_val)
    Purity = sum(egn_val.^2)

    #-----------------------------------
    return [Norm,Purity]
end

#   eigenvalue based generic properties of a density matrix
#-----------------------------------
function calc_SvN(density_matrix)
    rho = copy(density_matrix)

    egn_val = calc_Eigenvalues(rho)
    SvN = sum( [ - lam*log(lam) for lam in egn_val if lam > 0 ] )

    #-----------------------------------
    return SvN
end


calc_SvN (generic function with 1 method)

In [7]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------
J = 1 
B = 0.01

nsweeps = 20
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N = 10
#--------------------

site_list = collect(1:N) 
sites = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N ) 
#sites = siteinds("S=1/2", N)

H = Hamiltonian(sites) 

# ground state and first excited state
#--------------------------------------------
energy0,psi0 = dmrg(H,random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-6.822595169608869  maxlinkdim=8 maxerr=1.49E-15 time=28.855
After sweep 2 energy=-7.2494220217094725  maxlinkdim=32 maxerr=1.54E-16 time=0.028
After sweep 3 energy=-7.2531799346668695  maxlinkdim=64 maxerr=2.50E-16 time=0.148
After sweep 4 energy=-7.256499730363847  maxlinkdim=71 maxerr=6.78E-12 time=0.123
After sweep 5 energy=-7.2609847434024  maxlinkdim=72 maxerr=8.43E-12 time=0.107
After sweep 6 energy=-7.265757235709817  maxlinkdim=72 maxerr=3.10E-16 time=0.125
After sweep 7 energy=-7.270005974664683  maxlinkdim=72 maxerr=3.75E-16 time=0.118
After sweep 8 energy=-7.273711180974164  maxlinkdim=72 maxerr=4.45E-16 time=0.106
After sweep 9 energy=-7.277235985196487  maxlinkdim=72 maxerr=5.64E-16 time=0.116
After sweep 10 energy=-7.2808619246658415  maxlinkdim=72 maxerr=4.53E-16 time=0.119
After sweep 11 energy=-7.28456784347731  maxlinkdim=72 maxerr=2.25E-16 time=0.116
After sweep 12 energy=-7.28805743693939  maxlinkdim=72 maxerr=1.83E-16 time=0.109
After sweep 13

In [8]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1 
B = 0.01

nsweeps = 20
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N1 = 15
#--------------------

site_list1 = collect(1:N1) 

sites1 = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N1 ) 

H1 = Hamiltonian(sites1) 

# ground state and first excited state
#--------------------------------------------
energy01,psi01 = dmrg(H1,random_mps(sites1);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-10.55028210791221  maxlinkdim=8 maxerr=1.12E-15 time=0.032
After sweep 2 energy=-10.656850256952263  maxlinkdim=32 maxerr=4.58E-16 time=0.073
After sweep 3 energy=-10.717651352969778  maxlinkdim=124 maxerr=9.57E-12 time=0.374
After sweep 4 energy=-10.722216411912031  maxlinkdim=271 maxerr=9.90E-12 time=1.820
After sweep 5 energy=-10.723459514010546  maxlinkdim=315 maxerr=9.96E-12 time=3.969
After sweep 6 energy=-10.72374641835569  maxlinkdim=276 maxerr=9.81E-12 time=4.004
After sweep 7 energy=-10.723832609135261  maxlinkdim=247 maxerr=9.88E-12 time=2.970
After sweep 8 energy=-10.723867318790253  maxlinkdim=223 maxerr=9.90E-12 time=2.267
After sweep 9 energy=-10.723884236930877  maxlinkdim=205 maxerr=1.00E-11 time=1.792
After sweep 10 energy=-10.723893643489602  maxlinkdim=190 maxerr=1.00E-11 time=1.596
After sweep 11 energy=-10.723899249663075  maxlinkdim=179 maxerr=9.80E-12 time=1.423
After sweep 12 energy=-10.723902465136183  maxlinkdim=169 maxerr=9.95E-12 time=

In [9]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1 
B = 0.01

nsweeps = 20
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N2 = 20
#--------------------

site_list2 = collect(1:N2) 

sites2 = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N2 ) 

H2 = Hamiltonian(sites2) 

# ground state and first excited state
#--------------------------------------------
energy02,psi02 = dmrg(H2,random_mps(sites2);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-14.245726005971632  maxlinkdim=8 maxerr=1.24E-15 time=0.023
After sweep 2 energy=-14.378529476671178  maxlinkdim=32 maxerr=7.25E-12 time=0.114
After sweep 3 energy=-14.538839601104003  maxlinkdim=113 maxerr=9.86E-12 time=0.657
After sweep 4 energy=-14.545687505477217  maxlinkdim=286 maxerr=9.96E-12 time=4.952
After sweep 5 energy=-14.55221443096996  maxlinkdim=538 maxerr=9.99E-12 time=20.858
After sweep 6 energy=-14.558067188515553  maxlinkdim=670 maxerr=9.99E-12 time=39.314
After sweep 7 energy=-14.563115238468056  maxlinkdim=694 maxerr=9.98E-12 time=45.672
After sweep 8 energy=-14.56751015668422  maxlinkdim=691 maxerr=9.99E-12 time=47.123
After sweep 9 energy=-14.571336549221177  maxlinkdim=681 maxerr=9.95E-12 time=48.858
After sweep 10 energy=-14.574652876352296  maxlinkdim=668 maxerr=9.98E-12 time=44.764
After sweep 11 energy=-14.577501894594045  maxlinkdim=655 maxerr=9.97E-12 time=43.257
After sweep 12 energy=-14.579916164886509  maxlinkdim=642 maxerr=9.97E-1

In [10]:
#define the parameters of the hamiltonian and constuct MPO
#----------------------------------------------------------------------------

J = 1 
B = 0.01

nsweeps = 20
maxdim = [100, 200, 500, 1000, 2000, 5000, 10000, 25580]
cutoff = [1E-11]

#--------------------
N3 = 30
#--------------------

site_list3 = collect(1:N3) 

sites3 = siteinds( k->isodd(k) ? "S=1/2" : "S=1", N3 ) 

H3 = Hamiltonian(sites3) 

# ground state and first excited state
#--------------------------------------------
energy03,psi03 = dmrg(H3,random_mps(sites3);nsweeps,maxdim,cutoff,ishermitian=true) ;
#energy1,psi1 = dmrg(H,[psi0],random_mps(sites);nsweeps,maxdim,cutoff,ishermitian=true) ;

After sweep 1 energy=-21.438260517215056  maxlinkdim=8 maxerr=1.27E-15 time=0.086
After sweep 2 energy=-21.610207948507934  maxlinkdim=32 maxerr=8.74E-12 time=0.225
After sweep 3 energy=-21.854307671475407  maxlinkdim=110 maxerr=9.98E-12 time=1.001
After sweep 4 energy=-21.866759587746255  maxlinkdim=284 maxerr=9.99E-12 time=8.821
After sweep 5 energy=-21.871433669314076  maxlinkdim=439 maxerr=9.99E-12 time=39.867
After sweep 6 energy=-21.874501747207248  maxlinkdim=511 maxerr=9.98E-12 time=54.874
After sweep 7 energy=-21.876901774112028  maxlinkdim=537 maxerr=1.00E-11 time=59.236
After sweep 8 energy=-21.878839097997865  maxlinkdim=552 maxerr=1.00E-11 time=63.225
After sweep 9 energy=-21.880452950939187  maxlinkdim=559 maxerr=9.99E-12 time=59.849
After sweep 10 energy=-21.881789281876497  maxlinkdim=566 maxerr=9.99E-12 time=59.223
After sweep 11 energy=-21.88286848058856  maxlinkdim=564 maxerr=1.00E-11 time=58.003
After sweep 12 energy=-21.88367496244008  maxlinkdim=555 maxerr=9.99E-1

In [12]:
using Plots
using LaTeXStrings

seps=0:5
enta= []
enta1= []
enta2= []
enta3= []
A = 1
    println("Mixed spin chain 1/2-1 for N=10")
for sep in seps
    D =  A+2+sep
    ent = calc_LogNeg_4site(psi0,[A,D])
    push!(enta, ent)
    println(f"{sep} \t {ent}")
end
println("----------------------------")
println("----------------------------")

    println("Mixed spin chain 1/2-1 for N=15")
for sep in seps
    D =  A+2+sep
    ent1 = calc_LogNeg_4site(psi01,[A,D])
    push!(enta1, ent1)
    println(f"{sep} \t {ent1}")
end
println("----------------------------")
println("----------------------------")

    println("Mixed spin chain 1/2-1 for N=20")
for sep in seps
    D =  A+2+sep
    ent2 = calc_LogNeg_4site(psi02,[A,D])
    push!(enta2, ent2)
    println(f"{sep} \t {ent2}")
end
println("----------------------------")
println("----------------------------")


    println("Mixed spin chain 1/2-1 for N=30")
for sep in seps
    D =  A+2+sep
    ent3 = calc_LogNeg_4site(psi03,[A,D])
    push!(enta3, ent3)
    println(f"{sep} \t {ent3}")
end



plot(seps, enta, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="N=10", marker=:circle, color="red", lw=2)
plot!(seps, enta1, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="N=15", marker=:circle, color="blue", lw=2)
plot!(seps, enta2, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="N=20", marker=:circle, color="yellow", lw=2)
plot!(seps, enta3, xlabel= "Separation", ylabel=L"\mathcal{N}_4", label="N=30", marker=:circle, color="green", lw=2)

Mixed spin chain 1/2-1 for N=10
0 	 0.5565468869361281
1 	 0.28339023039886563
2 	 0.19498776789434405
3 	 0.1978500483704699
4 	 0.19573346167120215
5 	 0.23149293219310846
----------------------------
----------------------------
Mixed spin chain 1/2-1 for N=15
0 	 0.7178317460816431
1 	 0.439803492624339
2 	 0.25470289282165187
3 	 0.17489548001366564
4 	 0.07880839526299215
5 	 0.0
----------------------------
----------------------------
Mixed spin chain 1/2-1 for N=20
0 	 0.5548845310227205
1 	 0.2805983997251751
2 	 0.12799552015490784
3 	 0.08420859877280132
4 	 0.08127721295738756
5 	 0.07318091847719067
----------------------------
----------------------------
Mixed spin chain 1/2-1 for N=30
0 	 0.5548924816554837
1 	 0.2806047496509113
2 	 0.1267682521732197
3 	 0.0
4 	 0.0
5 	 0.0
